<a href="https://colab.research.google.com/github/praffuln/langchain/blob/master/langchain-2/langchain_10_pineconePDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install langchain
!pip install openAI
!pip install wikipedia
!pip install huggingface_hub
!pip install InstructorEmbedding
!pip install google-search-results
!pip install unstructured
!pip install libmagic
!pip install python-magic
!pip install python-magic-bin
#Install faiss Packages
!pip install faiss-cpu
!pip install sentence-transformers
!pip install wolframalpha
!pip install pypdf
!pip install youtube-transcript-api
!pip install pytube
!pip install python-dotenv
!pip install pinecone-client

ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 6.8 MB/s eta 0:00:00


# API

In [22]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV', '') # You may need to switch with your env


In [13]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os

load_dotenv()


False

# Load your data

Next let's load up some data. I've put a few 'loaders' on there which will load data from different locations. Feel free to use the one that suits you. The default one queries one of Paul Graham's essays for a simple example. This process will only stage the loader, not actually load it.



In [14]:
loader = TextLoader(file_path="/content/sample_data/vb.txt")

## Other options for loaders
# loader = PyPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
data = loader.load()


In [15]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')


You have 1 document(s) in your data
There are 9155 characters in your sample document
Here is a sample: January 2016Life is short, as everyone knows. When I was a kid I used to wonder
about this. Is life actually short, or are we really complaining
about its finiteness?  Would we be just as likely to fe


# **Chunk data data up into smaller documents**

While we could pass the entire essay to a model w/ long context, we want to be picky about which information we share with our model. The better signal to noise ratio we have the more likely we are to get the right answer.

The first thing we'll do is chunk up our document into smaller pieces. The goal will be to take only a few of those smaller pieces and pass them to the LLM.



In [16]:
# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)


In [17]:
# Let's see how many small chunks we have
print (f'Now you have {len(texts)} documents')

Now you have 20 documents


# Create embeddings of your documents to get ready for semantic search
Next up we need to prepare for similarity searches. The way we do this is through embedding our documents (getting a vector per document).

This will help us compare documents later on.



In [20]:
from langchain.vectorstores import Chroma, Pinecone
import pinecone


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [23]:
# Turn your texts into embeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


# Option #2: Pinecone (for cloud)

If you want to use pinecone, run the code below, if not then skip over to Chroma below it. You must go to Pinecone.io and set up an account



In [27]:
# # initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchaintest" # put in the name of your pinecone index here

docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)


# Query those docs to get your answer back

Great, those are just the docs which should hold our answer. Now we can pass those to a LangChain chain to query the LLM.

We could do this manually, but a chain is a convenient helper for us.



In [28]:
from langchain.chains.question_answering import load_qa_chain


In [29]:
# Load up your LLM
from langchain import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature": 0.5}
)

chain = load_qa_chain(llm, chain_type="stuff")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [31]:
query = "What is great about having kids?"
docs = docsearch.similarity_search(query)
docs

[Document(page_content="January 2016Life is short, as everyone knows. When I was a kid I used to wonder\nabout this. Is life actually short, or are we really complaining\nabout its finiteness?  Would we be just as likely to feel life was\nshort if we lived 10 times as long?Since there didn't seem any way to answer this question, I stopped\nwondering about it.  Then I had kids.  That gave me a way to answer"),
 Document(page_content="the question, and the answer is that life actually is short.Having kids showed me how to convert a continuous quantity, time,\ninto discrete quantities. You only get 52 weekends with your 2 year\nold.  If Christmas-as-magic lasts from say ages 3 to 10, you only\nget to watch your child experience it 8 times.  And while it's\nimpossible to say what is a lot or a little of a continuous quantity\nlike time, 8 is not a lot of something.  If you had a handful of 8"),
 Document(page_content='jabs into your consciousness like a pin.The things that matter aren\'t n

In [32]:
chain.run(input_documents=docs, question=query)


'They make you spend time on things that matter.'